In [154]:
import json
import pandas as pd
import numpy as np
import requests

ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "EUR_USD"} 
headers = {"Authorization": "Bearer "+ API_TOKEN}

response = requests.get("https://"+API_URL+PRICING_PATH, headers=headers, params=query) 

response.json()

{'time': '2024-08-04T22:39:47.960432872Z',
 'prices': [{'type': 'PRICE',
   'time': '2024-08-04T22:39:47.787628734Z',
   'bids': [{'price': '1.09039', 'liquidity': 500000},
    {'price': '1.09038', 'liquidity': 2500000},
    {'price': '1.09037', 'liquidity': 2000000},
    {'price': '1.09036', 'liquidity': 5000000},
    {'price': '1.09033', 'liquidity': 10000000},
    {'price': '1.09030', 'liquidity': 10000000}],
   'asks': [{'price': '1.09056', 'liquidity': 500000},
    {'price': '1.09057', 'liquidity': 500000},
    {'price': '1.09058', 'liquidity': 2000000},
    {'price': '1.09059', 'liquidity': 2000000},
    {'price': '1.09060', 'liquidity': 5000000},
    {'price': '1.09062', 'liquidity': 10000000},
    {'price': '1.09065', 'liquidity': 10000000}],
   'closeoutBid': '1.09030',
   'closeoutAsk': '1.09065',
   'status': 'tradeable',
   'tradeable': True,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}]}

In [155]:
import time

from_time = time.mktime(pd.to_datetime("01/01/2019").timetuple())
to_time = time.mktime(pd.to_datetime("01/02/2019").timetuple())


print(f"Window start: {from_time}, window end: {to_time}." )

header = {"Authorization": "Bearer "+API_TOKEN} 
query = {"from": str(from_time), "to": str(to_time), "granularity": "M1"}


INSTRUMENT = "GBP_JPY"

CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"


response = requests.get("https://"+API_URL+CANDLES_PATH, headers=header, params=query)


def json_to_pandas(json):
    json_file = json.json()
    price_json = json_file["candles"]
    dateTimes = []
    close_price, high_price, low_price, open_price= [], [], [], []
    volumes = []

    for candle in price_json:
        volumes.append(candle["volume"])
        dateTimes.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
        
    dataFrame = pd.DataFrame({"DateTime": dateTimes,
                              "Open": open_price,"High": high_price,
                              "Low": low_price,"Close": close_price,
                              "Volume": volumes})
    dataFrame['DateTime'] = pd.to_datetime(dataFrame['DateTime'])
    dataFrame.set_index('DateTime', inplace=True)

    return dataFrame

GBPJPY_df = json_to_pandas(response)

GBPJPY_df.head()

Window start: 1546322400.0, window end: 1546408800.0.


,Open,High,Low,Close,Volume
DateTime,,,,,
2019-01-01 22:00:00+00:00,139.828,139.828,139.828,139.828,1
2019-01-01 22:01:00+00:00,139.781,139.822,139.744,139.822,6
2019-01-01 22:02:00+00:00,139.822,139.822,139.805,139.805,3
2019-01-01 22:04:00+00:00,139.783,139.783,139.760,139.760,4
2019-01-01 22:05:00+00:00,139.780,139.780,139.753,139.771,82


In [156]:
from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

gbp_jpy_df = oa.show_granularity()

gbp_jpy_df

,Description
Granularity Value,
S5,"5 second candlesticks, minute alignment"
S10,"10 second candlesticks, minute alignment"
S15,"15 second candlesticks, minute alignment"
S30,"30 second candlesticks, minute alignment"
M1,"1 minute candlesticks, minute alignment"
M2,"2 minute candlesticks, hour alignment"
M4,"4 minute candlesticks, hour alignment"
M5,"5 minute candlesticks, hour alignment"
M10,"10 minute candlesticks, hour alignment"


In [157]:
ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"


def test_get_historical_data(instrument, start_date, end_date, granularity='D' ,as_pandas=True):
        # Must import Necessary Libraries
        import time
        import requests
        import pandas as pd

        # Convert Dates to DateTime objects
        start_dateTime = pd.to_datetime(start_date)
        end_dateTime = pd.to_datetime(end_date)

        # Convert Start Date & End Date to a UNIX timestamp
        unix_start = time.mktime(start_dateTime.timetuple())
        unix_end = time.mktime(end_dateTime.timetuple())

        # Authorization Header
        auth_header = {"Authorization": "Bearer "+API_TOKEN} 

        # Query Parameters
        parameters = {"from": str(unix_start), 
                      "to": str(unix_end), 
                      "granularity": granularity}
        
        # Historical Candle Stick Data URL
        candle_url = f"/v3/accounts/{ACCOUNT_ID}/instruments/{instrument}/candles"

        # Request for Candle Stick Data
        json_response = requests.get("https://"+API_URL+candle_url, headers=auth_header, params=parameters).json()

        # Return as Pandas DataFrame
        if as_pandas is True:
            # Convert Json to Pandas df
            candle_df = pd.json_normalize(json_response['candles'])

            # Drop complete Column
            candle_df.drop('complete', inplace=True, axis=1)

            # Rename Columns
            candle_df.rename(inplace=True,columns={'time':'DateTime',
                                                   'volume':'Volume',
                                                   'mid.o':'Open',
                                                   'mid.h':'High',
                                                   'mid.l':'Low',
                                                   'mid.c':'Close'})

            # Convert Datetime= Column to datetime object
            candle_df['DateTime'] = pd.to_datetime(candle_df['DateTime'])

            # Set DateTime to Index
            candle_df.set_index('DateTime',inplace=True)

            # Restructure Column Names
            candle_df = candle_df[['Open','High','Low','Close','Volume']]

            return candle_df
        else: 
            return json_response

df = test_get_historical_data(instrument='EUR_USD', 
                            start_date='08/09/2020',
                            end_date='08/09/2021',
                            granularity='H2')
df

,Open,High,Low,Close,Volume
DateTime,,,,,
2020-08-09 21:00:00+00:00,1.17934,1.17934,1.17737,1.17762,1093
2020-08-09 23:00:00+00:00,1.17764,1.17956,1.17764,1.17937,2094
2020-08-10 01:00:00+00:00,1.17939,1.17984,1.17899,1.17931,2314
2020-08-10 03:00:00+00:00,1.17930,1.17964,1.17897,1.17910,1126
2020-08-10 05:00:00+00:00,1.17912,1.18008,1.17650,1.17662,6316
...,...,...,...,...,...
2021-08-06 19:00:00+00:00,1.17574,1.17646,1.17565,1.17616,2473
2021-08-08 21:00:00+00:00,1.17643,1.17643,1.17508,1.17544,1488
2021-08-08 23:00:00+00:00,1.17546,1.17552,1.17425,1.17508,3808


In [158]:
import time    
from datetime import datetime, timedelta
import math
def get_all_candles(start,end,gran):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    print(f'start Date:{start} & end Date:{end}')
    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'start DateTime:{start_dateTime} & end DateTime:{end_dateTime}')


    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 
    print(f'Seconds Between:{seconds_between}')

    if seconds_between > 4000:
        print('Greater than 4000 seconds')
        max_date = start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
        print(f'First Max date: {max_date}')

        # Create Pandas DataFrame
        can_df = pd.DataFrame()

        new_start_dateTime = start_dateTime
       
        while True:
            if max_date >= end_dateTime:
                print(f'{new_start_dateTime} - {end_dateTime}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                            start_date=new_start_dateTime,
                                            end_date=end_dateTime,
                                            granularity=gran)
                can_df = pd.concat([can_df, new_df])
                break

            else:
                print(f'{new_start_dateTime} - {max_date}')
                new_df = test_get_historical_data(instrument='EUR_USD', 
                                                start_date=new_start_dateTime,
                                                end_date=max_date,
                                                granularity=gran)
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                max_date = new_start_dateTime+timedelta(seconds=4000*math.floor(gran_secs.get(gran)))
                can_df = pd.concat([can_df, new_df])
                                        
        return can_df

master_df = get_all_candles(start='08/09/2023',
                            end='08/02/2024',
                            gran='M1')

master_df

start Date:08/09/2023 & end Date:08/02/2024
start DateTime:2023-08-09 00:00:00 & end DateTime:2024-08-02 00:00:00
Seconds Between:31017600.0
Greater than 4000 seconds
First Max date: 2023-08-11 18:40:00
2023-08-09 00:00:00 - 2023-08-11 18:40:00
2023-08-11 18:41:00 - 2023-08-14 13:21:00
2023-08-14 13:22:00 - 2023-08-17 08:02:00
2023-08-17 08:03:00 - 2023-08-20 02:43:00
2023-08-20 02:44:00 - 2023-08-22 21:24:00
2023-08-22 21:25:00 - 2023-08-25 16:05:00
2023-08-25 16:06:00 - 2023-08-28 10:46:00
2023-08-28 10:47:00 - 2023-08-31 05:27:00
2023-08-31 05:28:00 - 2023-09-03 00:08:00
2023-09-03 00:09:00 - 2023-09-05 18:49:00
2023-09-05 18:50:00 - 2023-09-08 13:30:00
2023-09-08 13:31:00 - 2023-09-11 08:11:00
2023-09-11 08:12:00 - 2023-09-14 02:52:00
2023-09-14 02:53:00 - 2023-09-16 21:33:00
2023-09-16 21:34:00 - 2023-09-19 16:14:00
2023-09-19 16:15:00 - 2023-09-22 10:55:00
2023-09-22 10:56:00 - 2023-09-25 05:36:00
2023-09-25 05:37:00 - 2023-09-28 00:17:00
2023-09-28 00:18:00 - 2023-09-30 18:58:00

,Open,High,Low,Close,Volume
DateTime,,,,,
2023-08-09 05:00:00+00:00,1.09670,1.09688,1.09670,1.09684,64
2023-08-09 05:01:00+00:00,1.09685,1.09687,1.09678,1.09678,69
2023-08-09 05:02:00+00:00,1.09680,1.09694,1.09678,1.09691,62
2023-08-09 05:03:00+00:00,1.09693,1.09694,1.09687,1.09688,40
2023-08-09 05:04:00+00:00,1.09692,1.09710,1.09692,1.09708,60
...,...,...,...,...,...
2024-08-02 04:55:00+00:00,1.07955,1.07968,1.07954,1.07966,52
2024-08-02 04:56:00+00:00,1.07967,1.07969,1.07964,1.07965,70
2024-08-02 04:57:00+00:00,1.07966,1.07972,1.07961,1.07972,93


In [162]:
import time
from datetime import datetime, timedelta
import math
import tqdm
def get_all_candles_upgraded(start,end,gran, verbose=False, pair='EUR_USD'):

    gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
                 'M1': 60,'M2': 120,'M4': 240,'M5': 300,
                 'M10': 600,'M15': 900,'M30': 1800,
                 'H1': 3600,'H2': 7200,'H3': 10800,
                 'H4': 14400,'H6': 21600,'H8': 28800,
                 'H12': 43200,'D': 86400,'W': 604800,
                 'M': 2620800}

    MAX_AMOUNT = 4000

    start_dateTime = pd.to_datetime(start)
    end_dateTime = pd.to_datetime(end)
    print(f'Gathering Data {start_dateTime} - {end_dateTime}')


    time_distance = end_dateTime - start_dateTime   
    seconds_between = time_distance.total_seconds() 

    if seconds_between > MAX_AMOUNT:
        max_date = start_dateTime+timedelta(seconds=MAX_AMOUNT*math.floor(gran_secs.get(gran)))

        # Create Pandas DataFrame
        can_df = pd.DataFrame()
        start_dateTime_list = []
        end_dateTime_list = []
        timeFrame_df = pd.DataFrame(columns=['Start DateTime','End DateTime'])

        new_start_dateTime = start_dateTime

        while True:
            if verbose:
                print(f'{new_start_dateTime} - {max_date}')

            if max_date >= end_dateTime:
                start_dateTime_list.append(new_start_dateTime)
                end_dateTime_list.append(end_dateTime)
                break

            else: 
                new_start_dateTime = max_date+timedelta(seconds=gran_secs.get(gran))
                start_dateTime_list.append(new_start_dateTime)
                max_date = new_start_dateTime+timedelta(seconds=MAX_AMOUNT*math.floor(gran_secs.get(gran)))

        for i in tqdm.tqdm(range(len(start_dateTime_list)),colour='GREEN', 
                           ncols=100,desc =f'Downloading {pair} {gran} Data'):

            new_df = test_get_historical_data(instrument='EUR_USD', 
                                    start_date=start_dateTime_list[i],
                                    end_date=end_dateTime_list[i],
                                    granularity=gran)
            can_df = pd.concat([can_df, new_df])
    return can_df

master_df = get_all_candles_upgraded(start='08/01/2023',
                                     end='08/01/2024',
                                     gran='M5',
                                     pair='EUR_USD')

master_df.to_csv('EUR-USD-M1.csv')

Gathering Data 2023-08-01 00:00:00 - 2024-08-01 00:00:00


KeyError: 'candles'

In [ ]:
range(10)

range(0, 10)